In [14]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime

In [7]:
# ensemble할 파일 가져오기
input_csv = list()
for file in glob.glob("../ensembles_inference/*.csv"):
    df = pd.read_csv(file)
    input_csv.append(df[["prediction"]])
csv_nums = len(input_csv)

In [8]:
# prediction 열의 value만 취하기
input_sum = list()
for df in input_csv:
    input_sum.append(list(df["prediction"].iloc[:744].values))

In [ ]:
output_np = np.array(input_sum).T # (test 문제 수 x  csv 개수) -> (csv 개수 x test 문제 수)

result = []
for row in output_np[:]: # 한 행(문제)씩 살핌
    print(type(row))
    ans = np.where(row > 0.5,1,0) # 0.5를 넘기면 1, 넘기지 못하면 0
    if ans.sum() > csv_nums//2: # ans에 sum을 취해 1의 개수 세기
        value = np.average(row[np.where(ans == 1)]) # 1의 개수가 더 많으면 0.5 보다 큰 확률의 평균
    elif ans.sum() < csv_nums//2:
        value = np.average(row[np.where(ans == 0)]) # 0의 개수가 더 많으면 0.5 보다 작은 확률의 평균
    else:
        value = np.average(row) # 1과 0의 개수가 동일한 경우 전체 평균
    result.append(value)
df['prediction'] = result

In [24]:
now = "_".join(str(datetime.now()).split())[:-10]
df.to_csv(f"../ensembles_submit/{now}_hs_ensemble.csv", index_label="id")